In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install clarifai_grpc
from PIL import Image
import os
import pandas as pd
%cd '/content'

In [ ]:
for f in os.listdir('data'):
  if f[-4:] == ".jpg":
    os.remove(os.path.join('data', f))

In [ ]:
for f in os.listdir('data'):
  if f[-4:] != ".ppm":
    continue
  im = Image.open(os.path.join('data', f))
  im_jpg = im.convert('RGB')
  im_jpg.save(os.path.join('data', f[:-4] + '.jpg'))

In [ ]:
from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
from clarifai_grpc.grpc.api.status import status_code_pb2

USER_ID = 'marsxiang5902'
APP_ID = 'HTN2022'
PAT = ''

In [ ]:
def uploadFile(IMAGE_FILE_LOCATION):
  channel = ClarifaiChannel.get_grpc_channel()
  stub = service_pb2_grpc.V2Stub(channel)

  metadata = (('authorization', 'Key ' + PAT),)

  userDataObject = resources_pb2.UserAppIDSet(user_id=USER_ID, app_id=APP_ID)

  with open(IMAGE_FILE_LOCATION, "rb") as f:
    file_bytes = f.read()

  post_inputs_response = stub.PostInputs(
    service_pb2.PostInputsRequest(
      user_app_id=userDataObject,
      inputs=[
        resources_pb2.Input(
          data=resources_pb2.Data(
            image=resources_pb2.Image(
              base64=file_bytes
            )
          )
        )
      ]
    ),
    metadata=metadata
  )


  if post_inputs_response.status.code != status_code_pb2.SUCCESS:
    print(post_inputs_response.status)
    raise Exception("Post inputs failed, status: " + post_inputs_response.status.description)

  return post_inputs_response.inputs[0].id

In [ ]:
def updateFile(name, INPUT_ID, CONCEPTS):
  channel = ClarifaiChannel.get_grpc_channel()
  stub = service_pb2_grpc.V2Stub(channel)

  metadata = (('authorization', 'Key ' + PAT),)

  userDataObject = resources_pb2.UserAppIDSet(user_id=USER_ID, app_id=APP_ID)

  im = Image.open(name)
  width, height = im.size

  post_annotations_response = stub.PostAnnotations(
      service_pb2.PostAnnotationsRequest(
          user_app_id=userDataObject,
          annotations=list(map(lambda tpl:
            resources_pb2.Annotation(
                input_id=INPUT_ID,
                data=resources_pb2.Data(
                    regions=[
                        resources_pb2.Region(
                            region_info=resources_pb2.RegionInfo(
                                bounding_box=resources_pb2.BoundingBox(
                                    left_col=tpl[1][0]/width,
                                    top_row=tpl[1][1]/height,
                                    right_col=tpl[1][2]/width,
                                    bottom_row=tpl[1][3]/height
                                )
                            ),
                            data=resources_pb2.Data(
                                concepts=[
                                    resources_pb2.Concept(id=str(tpl[0]), value=1.)
                                ]
                            )
                        )
                    ]
                )
            ), CONCEPTS)
          )
      ),
      metadata=metadata
  )

  if post_annotations_response.status.code != status_code_pb2.SUCCESS:
      print(post_annotations_response.status)
      raise Exception("Post annotations failed, status: " + post_annotations_response.status.description)

In [ ]:
df = pd.read_csv('data/gt.txt', sep=';', names=['path','left','top','right','bottom','id'])
df.head()

In [ ]:
mp = {}
for f in os.listdir('data'):
  if f[-4:] != ".jpg":
    continue
  mp[f] = []

In [ ]:
stop = '00100.ppm'
for i, r in df.iterrows():
  if r['path'] == stop:
    break
  mp[r['path'][:-4] + '.jpg'].append((r['id'], (r['left'], r['top'], r['right'], r['bottom'])))

In [ ]:
ids = []
for k, v in mp.items():
  ids.append(uploadFile(os.path.join('data', k)))
  if len(ids) % 10 == 0:
    print(len(ids))

In [ ]:
i = 0
for k, v in mp.items():
  updateFile(os.path.join('data', k), ids[i], v)
  i += 1
  if i % 10 == 0:
    print(i)

In [ ]:
def queryFile(path):
  channel = ClarifaiChannel.get_grpc_channel()
  stub = service_pb2_grpc.V2Stub(channel)

  metadata = (('authorization', 'Key ' + PAT),)

  userDataObject = resources_pb2.UserAppIDSet(user_id=USER_ID, app_id=APP_ID)


  with open(path, "rb") as f:
    file_bytes = f.read()

  post_model_outputs_response = stub.PostModelOutputs(
      service_pb2.PostModelOutputsRequest(
          user_app_id=userDataObject,
          model_id="GTSDB-Detector",
          inputs=[
              resources_pb2.Input(
                  data=resources_pb2.Data(
                      image=resources_pb2.Image(
                          base64=file_bytes
                      )
                  )
              )
          ]
      ),
      metadata=metadata
  )

  if post_model_outputs_response.status.code != status_code_pb2.SUCCESS:
      print("There was an error with your request!")
      print("\tCode: {}".format(post_model_outputs_response.outputs[0].status.code))
      print("\tDescription: {}".format(post_model_outputs_response.outputs[0].status.description))
      print("\tDetails: {}".format(respopost_model_outputs_responsense.outputs[0].status.details))
      raise Exception("Post model outputs failed, status: " + post_model_outputs_response.status.description)

  # Since we have one input, one output will exist here.
  output = post_model_outputs_response.outputs[0]

  return [(concept.name, concept.value) for concept in output.data.concepts]

Predicted concepts:
